In [ ]:
cd /home/yuchen/pulse2percept

In [ ]:
import pulse2percept as p2p
from pulse2percept.implants import ArgusII
import shapes
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import string
import skimage
from skimage.measure import label, regionprops, regionprops_table
from pulse2percept.viz import plot_argus_phosphenes
import math
from statistics import mean
from sklearn.linear_model import LinearRegression
import statsmodels.api as sm 
import pingouin as pg
from statsmodels.stats.outliers_influence import variance_inflation_factor

## Code

### area

In [ ]:
# load data
data = shapes.load_shapes("/home/yuchen/shapes/data/shapes.h5", subjects=['12-005','51-009','52-001'],stim_class=None)
data = data[data['electrode2'] == str()].reset_index(drop=True)

df_total = pd.DataFrame()
df_test = pd.DataFrame()
df_AllSub = pd.DataFrame()
df_k =pd.DataFrame({})
sub = ['12-005','51-009','52-001']

for subj in sub:
    df = data[['subject','amp1','freq','electrode1']].drop_duplicates()
    df = df[df.subject == subj].reset_index(drop=True)
    lst = []
    lst_area = []
    for i in range(len(df)):
        df_temp = data[(data.amp1 == df.amp1[i]) & (data.freq == df.freq[i]) & (data.subject == df.subject[i]) & (data.electrode1 == df.electrode1[i])].reset_index(drop=True)
        num = len(df_temp[df_temp.num_regions > 1]) / len(df_temp)  # compute the % of trials that had multiple phosphenes 
        lst.append(num)
        if num == 0:
            lst_area.append(df_temp.area.mean())

        # if there are more than one phosphene in a drawing, pick the phosphene whose size, orientation, and eccentricity is closer to the average size, orientation, and eccentricity in that group
        elif num <= 0.5:
            avg_area = df_temp[df_temp.num_regions == 1][['area']].mean().tolist()[0]
            avg_orientation = df_temp[df_temp.num_regions == 1][['orientation']].mean().tolist()[0]
            avg_eccentricity = df_temp[df_temp.num_regions == 1][['eccentricity']].mean().tolist()[0]
            for item in range(len(df_temp)):
                if df_temp.num_regions[item] > 1:
                        label_img = skimage.measure.label(df_temp['image'][item]>0)
                        regions = regionprops(label_img)
                        props = regionprops_table(label_img, properties=('centroid',
                                                                         'orientation',
                                                                         'eccentricity',
                                                                         'major_axis_length',
                                                                         'minor_axis_length',
                                                                         'area'))
                        df_similar = pd.DataFrame(props).astype('object')
                        area_id = 0
                        area_compare = 9999999
                        for j in range(len(df_similar)):
                            result_temp = (df_similar.iloc[:, 6].tolist()[j] - avg_area)**2 + (df_similar.iloc[:, 2].tolist()[j] - avg_orientation)**2 + (df_similar.iloc[:, 3].tolist()[j] - avg_eccentricity)**2
                            if result_temp < area_compare:
                                area_id = j
                                area_compare = result_temp
                        df_temp.at[item,'area'] = df_similar.iloc[:, 6].tolist()[area_id]

            lst_area.append(df_temp.area.mean())
        else:
            df_k = pd.concat([df_k,df_temp])
            lst_area.append(0)

    df['helper'] = lst
    df['area'] = lst_area
    df = df[df.helper <= 0.5].reset_index(drop = True)  # remove groups with >50% of drawings were multi-phopshene drawings
    df = df.drop(columns = ['helper'])
    df = df.groupby(['subject','amp1','freq','electrode1']).mean().reset_index()

    lst = []
    lst_dtf = []
    lst_d = []
    upper = []
    
    # add electrode-retina distance
    if subj == '12-005':
        lst_d = [0,0,0,2,7,7,0,0,5,0,
                0,0,4,11,13,15,15,3,7,4,
                0,0,15,16,17,17,19,16,9,4,
                0,0,16,19,15,17,22,25,13,10,
                0,0,8,15,14,13,17,23,14,2,
                0,0,10,15,13,12,9,11,5,-1]  
    else:
        lst_d = [0] * 60
    s2 = shapes.subject_params[subj]
    implant,model = shapes.model_from_params(s2)
    for i in string.ascii_uppercase[0:6]: 
        for j in range(1,11):
            electrode = i + str(j)
            lst.append(electrode)
            # add electrode-fovea distance
            lst_dtf.append(math.sqrt(implant[electrode].x**2 +implant[electrode].y**2 ))
            
            e1_x = implant[electrode].x
            e1_y = implant[electrode].y
    df_o = pd.DataFrame(lst, columns=['electrode1'])
    df_o['distance_to_fovea'] = lst_dtf
    df_o['distance_to_implant'] = lst_d
    
    if subj == '12-005':
        lst = lst[:-1]
    # add electrode-retina distance and remove electrodes without data
    elif subj == '52-001':
        lst = ['F1','F2','F4','F5','F6','F7','F8','F9','F10',
               'E1','E2','E3','E4','E5','E6','E7','E8','E9','E10', 
               'D1','D2','D3','D4','D5','D6','D7','D8','D9','D10', 
               'C3','C4','C5','C6','C7','C8','C9','C10', 
               'B1','B3','B4','B5','B6','B7','B8','B9','B10', 
               'A1','A2','A3','A4','A5','A6','A7','A8','A9','A10']
    else:
        lst = ['F1','F2','F3','F4','F5','F6','F7','F8','F9','F10',
               'E1','E2','E3','E4','E5','E6','E7','E8','E9','E10', 
               'D1','D2','D3','D4','D5','D6','D7','D8','D9','D10', 
               'C2','C3','C4','C5','C6','C7','C8','C9','C10', 
               'B3','B4','B5','B6','B7','B8','B9','B10', 
               'A3','A4','A5','A6','A7','A8','A9','A10']
        
    df_o = df_o[df_o.electrode1.isin(lst)]
    df_o_temp = df_o.copy()
    df_o_temp['subject'] = subj
    
    df_test = pd.concat([df_test, df_o_temp])
    
    df = df_o.merge(df, how = 'inner', on = 'electrode1')
    
    df_reference = df[(df.subject == subj) & (df.amp1 == 2) & (df.freq == 20) & 
                      (df.distance_to_implant == 0) & 
                      (df.distance_to_fovea>2950) & (df.distance_to_fovea<3450)].reset_index(drop=True)
    reference_number  = df_reference.area.mean()
    for i in range(len(df)):
        df.at[i, 'area'] = df.at[i, 'area'] / reference_number
        
    df_total = pd.concat([df_total, df])

for i in sub:
    temp = df_total[df_total.subject == i].reset_index(drop=True)
    temp.to_csv('/home/yuchen/shapes/notebooks/' + i + '.csv')
df_total.to_csv('/home/yuchen/shapes/notebooks/out.csv')

In [ ]:
df_test

In [ ]:
len(df2)

In [ ]:
from scipy import stats
df1 = df_test[df_test['subject']=='51-009'][['electrode1','distance_to_fovea']]
df2 = df_test[df_test['subject']=='52-001'][['electrode1','distance_to_fovea']]

stats.ttest_ind(df2.distance_to_fovea, df1.distance_to_fovea, equal_var=False, alternative='greater')

In [ ]:
from scipy import stats
df1 = df_test[df_test['subject']=='12-005'][['electrode1','distance_to_fovea']].drop_duplicates()
df2 = df_test[df_test['subject']=='51-009'][['electrode1','distance_to_fovea']].drop_duplicates()

stats.ttest_ind(df1.distance_to_fovea, df2.distance_to_fovea, equal_var=False, alternative='greater')

In [ ]:
df1.drop_duplicates()['distance_to_fovea'].tolist()

In [ ]:
copy = df_total.copy()
df_AllSub = df_total.copy()
df_s = pd.DataFrame({})
for subj in ['12-005','51-009','52-001','All']:
    if subj=='All':
        df_t = df_AllSub.copy()
    else:   
        df_t = copy[copy.subject == subj]
    df_t['amp1'] = (df_t['amp1'] - df_t['amp1'].mean()) / df_t['amp1'].std()
    df_t['freq'] = (df_t['freq'] - df_t['freq'].mean()) / df_t['freq'].std()
    df_t['distance_to_implant'] = (df_t['distance_to_implant'] - df_t['distance_to_implant'].mean()) / df_t['distance_to_implant'].std()
    df_t['distance_to_fovea'] = (df_t['distance_to_fovea'] - df_t['distance_to_fovea'].mean()) / df_t['distance_to_fovea'].std()
    if subj == 'All':  
        df_AllSub = df_t.copy()
    else:
        df_s = pd.concat([df_s, df_t])
        df_s = df_s.fillna(0)
        
df_total = df_s.copy().reset_index(drop=True)

In [ ]:
fig, axs= plt.subplots(ncols=4, figsize=(56, 14))
column_lst = ['amp1', 'freq','distance_to_fovea', 'distance_to_implant']
name_lst = ['Amplitude','Frequency','Foveal Eccentricity', 'Electrode-Implant Distance']
for dv in range(4):
    reg = LinearRegression().fit(df_total[column_lst[:dv] + column_lst[dv+1 :]], df_total[['area']])
    y_predicted = reg.predict(df_total[column_lst[:dv] + column_lst[dv+1 :]])
    y = df_total[['area']]-y_predicted

    reg = LinearRegression().fit(df_total[column_lst[:dv] + column_lst[dv+1 :]], df_total[column_lst[dv]])
    y_predicted = reg.predict(df_total[column_lst[:dv] + column_lst[dv+1 :]])
    x = df_total[column_lst[dv]]-y_predicted
    df_total['x'] = x
    df_total['y'] = y
    subject = ['12-005','51-009','52-001']
    marker_lst = ['o','s','^']
    color_lst = ['#1E88E5', '#FFC107','#004D40']
    for i in range(len(subject)):
        temp = df_total[df_total.subject == subject[i]]
        axs[dv].plot(temp['x'],temp['y'],marker_lst[i], color=color_lst[i],alpha=0.6,markersize=26)
    axs[dv].legend(subject, loc='upper right',prop={'size': 30})
    reg = LinearRegression().fit(np.array(x).reshape(-1,1),np.array(y).reshape(-1,1))
    y_pred = reg.predict(np.array(x).reshape(-1,1))
    axs[dv].plot(x, y_pred,'-', color="black",linewidth=2)
    axs[dv].set(xlabel = name_lst[dv], ylabel = 'Area',ylim=(-2,4))
    for item in ([axs[dv].xaxis.label, axs[dv].yaxis.label]+axs[dv].get_xticklabels()+axs[dv].get_yticklabels()):
        item.set_fontsize(30)
fig.savefig('/home/yuchen/paper/11a. Single-Electrode Area.pdf', transparent=True)

In [ ]:
for subj in ['12-005','51-009','52-001','AllSubjects']:
    temp = df_total[df_total.subject == subj].reset_index(drop=True)
    if subj == '12-005':
        X = temp[['amp1','freq','distance_to_fovea','distance_to_implant']]
    elif subj == 'AllSubjects':
        temp = df_AllSub.copy()
        X = temp[['amp1','freq','distance_to_fovea','distance_to_implant']]
    else:
        X = temp[['amp1','freq','distance_to_fovea']]
    y = temp.area
    X2 = sm.add_constant(X)
    est = sm.OLS(y, X2)
    est2 = est.fit()
    print('subject: ' + subj)
    print(est2.summary())
    print('\ncoefficient with more digits: ')
    print(est2.params)
    print('\nvariance inflation factor: ')
    print([[X.columns[i],variance_inflation_factor(X.values, i)]
                          for i in range(len(X.columns))])
    print('\npartial correlation: ')
    X['area'] = temp['area']
    print(X.pcorr())
    print(' ')

In [ ]:
import scipy
import scipy.stats as spst
def scatter_correlation(x, y, marker='o', color='k', ax=None, autoscale=True, text = True,scatter=True):
    # If data are Pandas series, use their names to label the axes:
    x_label = x.name if hasattr(x, 'name') else ''
    y_label = y.name if hasattr(y, 'name') else ''
    x = np.asarray(x)
    y = np.asarray(y)
    # Ignore NaN:
    isnan = np.isnan(x) | np.isnan(y)
    x = x[~isnan]
    y = y[~isnan]
    if not np.all(x.shape == y.shape):
        raise ValueError("x and y must have the same shape")
    if len(x) < 2:
        raise ValueError("x and y must at least have 2 data points.")
    # Scatter plot the data:
    if ax is None:
        ax = plt.gca()
    if scatter is True:
        ax.scatter(x, y, marker=marker, s=50, c=color, edgecolors='w', alpha=0.5)
    # Fit the regression curve:
    slope, intercept, rval, pval, _ = spst.linregress(x, y)
    def fit(x): return slope * x + intercept
    ax.plot([np.min(x), np.max(x)], [fit(np.min(x)), fit(np.max(x))], 'k--', c=color)
    # Annotate with fitting results:
    if text == True:
        pval = ("%.2e" % pval) if pval < 0.001 else ("%.03f" % pval)
        a = ax.axis()
        ax.text(a[1], a[2], "$N$=%d\n$r$=%.3f, $p$=%s" % (len(y), rval, pval),
                va='bottom', ha='right')
    ax.set_xlabel(x_label)
    ax.set_ylabel(y_label)
    if autoscale:
        ax.autoscale(True)
    return ax

### eccentricity

In [ ]:
data = shapes.load_shapes("/home/yuchen/shapes/data/shapes.h5", subjects=['12-005','51-009','52-001'],stim_class=None)
data = data[data['electrode2'] == str()].reset_index(drop=True)

df_total = pd.DataFrame()
sub = ['12-005','51-009','52-001']

for subj in sub:
    df = data[['subject','amp1','freq','electrode1']].drop_duplicates()
    df = df[df.subject == subj].reset_index(drop=True)
    lst = []
    lst_area = []
    for i in range(len(df)):
        df_temp = data[(data.amp1 == df.amp1[i]) & (data.freq == df.freq[i]) & (data.subject == df.subject[i]) & (data.electrode1 == df.electrode1[i])].reset_index(drop=True)
        num = len(df_temp[df_temp.num_regions > 1]) / len(df_temp)
        lst.append(num)
        if num == 0:
            lst_area.append(df_temp.eccentricity.mean())

        # if there are more than one phosphene in a drawing, pick the phosphene whose size, orientation, and eccentricity is closer to the average size, orientation, and eccentricity in that group
        elif num <= 0.5:
            avg_area = df_temp[df_temp.num_regions == 1][['area']].mean().tolist()[0]
            avg_orientation = df_temp[df_temp.num_regions == 1][['orientation']].mean().tolist()[0]
            avg_eccentricity = df_temp[df_temp.num_regions == 1][['eccentricity']].mean().tolist()[0]
            for item in range(len(df_temp)):
                if df_temp.num_regions[item] > 1:
                        label_img = skimage.measure.label(df_temp['image'][item]>0)
                        regions = regionprops(label_img)
                        props = regionprops_table(label_img, properties=('centroid',
                                                                         'orientation',
                                                                         'eccentricity',
                                                                         'major_axis_length',
                                                                         'minor_axis_length',
                                                                         'area'))
                        df_similar = pd.DataFrame(props).astype('object')
                        area_id = 0
                        area_compare = 9999999
                        for j in range(len(df_similar)):
                            result_temp = (df_similar.iloc[:, 6].tolist()[j] - avg_area)**2 + (df_similar.iloc[:, 2].tolist()[j] - avg_area)**2 + (df_similar.iloc[:, 3].tolist()[j] - avg_area)**2
                            if result_temp < area_compare:
                                area_id = j
                                area_compare = result_temp
                        df_temp.at[item,'eccentricity'] = df_similar.iloc[:, 3].tolist()[area_id]
            lst_area.append(df_temp.eccentricity.mean())
        else:
            lst_area.append(0)

    df['helper'] = lst
    df['area'] = lst_area
    df = df[df.helper <= 0.5].reset_index(drop = True)
    df = df.drop(columns = ['helper'])
    df = df.groupby(['subject','amp1','freq','electrode1']).mean().reset_index()


    lst = []
    lst_dtf = []
    lst_d = []
    upper = []
    
    if subj == '12-005':
        lst_d = [0,0,0,2,7,7,0,0,5,0,
                0,0,4,11,13,15,15,3,7,4,
                0,0,15,16,17,17,19,16,9,4,
                0,0,16,19,15,17,22,25,13,10,
                0,0,8,15,14,13,17,23,14,2,
                0,0,10,15,13,12,9,11,5,-1]  # only 59 elements bc F10's distance is unidentifiable
    else:
        lst_d = [0] * 60
    s2 = shapes.subject_params[subj]
    implant,model = shapes.model_from_params(s2)
    for i in string.ascii_uppercase[0:6]: 
        for j in range(1,11):
            electrode = i + str(j)
            lst.append(electrode)
            lst_dtf.append(math.sqrt(implant[electrode].x**2 +implant[electrode].y**2 ))
            
            if implant[electrode].y > 0:
                upper.append(1)
            else:
                upper.append(0)
            e1_x = implant[electrode].x
            e1_y = implant[electrode].y
    df_o = pd.DataFrame(lst, columns=['electrode1'])
    df_o['distance_to_fovea'] = lst_dtf
    df_o['distance_to_implant'] = lst_d
    
    if subj == '12-005':
        lst = lst[:-1]
    elif subj == '52-001':
        lst = ['F1','F2','F4','F5','F6','F7','F8','F9','F10',
               'E1','E2','E3','E4','E5','E6','E7','E8','E9','E10', 
               'D1','D2','D3','D4','D5','D6','D7','D8','D9','D10', 
               'C3','C4','C5','C6','C7','C8','C9','C10', 
               'B1','B3','B4','B5','B6','B7','B8','B9','B10', 
               'A1','A2','A3','A4','A5','A6','A7','A8','A9','A10']
        
    df_o = df_o[df_o.electrode1.isin(lst)]
    
    df = df_o.merge(df, how = 'inner', on = 'electrode1')
    
    df_reference = df[(df.subject == subj) & (df.amp1 == 2) & (df.freq == 20) & (df.distance_to_implant == 0)].reset_index(drop=True)
    reference_number  = df_reference.area.mean()
    for i in range(len(df)):
        df.at[i, 'area'] = df.at[i, 'area'] / reference_number
        

    df_total = pd.concat([df_total, df])
    
    

for i in sub:
    temp = df_total[df_total.subject == i].reset_index(drop=True)
    temp.to_csv('/home/yuchen/shapes/notebooks/' + i + '.csv')
df_total.to_csv('/home/yuchen/shapes/notebooks/out.csv')

In [ ]:
fig, axs= plt.subplots(ncols=4, figsize=(28, 7))
column_lst = ['amp1', 'freq','distance_to_fovea', 'distance_to_implant']
name_lst = ['Amplitude','Frequency','Foveal Eccentricity', 'Electrode-Implant Distance']
for dv in range(4):
    reg = LinearRegression().fit(df_total[column_lst[:dv] + column_lst[dv+1 :]], df_total[['area']])
    y_predicted = reg.predict(df_total[column_lst[:dv] + column_lst[dv+1 :]])
    y = df_total[['area']]-y_predicted

    reg = LinearRegression().fit(df_total[column_lst[:dv] + column_lst[dv+1 :]], df_total[column_lst[dv]])
    y_predicted = reg.predict(df_total[column_lst[:dv] + column_lst[dv+1 :]])
    x = df_total[column_lst[dv]]-y_predicted
    df_total['x'] = x
    df_total['y'] = y
    subject = ['12-005','51-009','52-001']
    marker_lst = ['o','X','s']
    color_lst = ['#1E88E5', '#FFC107','#004D40']
    for i in range(len(subject)):
        temp = df_total[df_total.subject == subject[i]]
        axs[dv].plot(temp['x'],temp['y'],marker_lst[i], color=color_lst[i],alpha=0.8)
    axs[dv].legend(subject, loc='upper right')
    reg = LinearRegression().fit(np.array(x).reshape(-1,1),np.array(y).reshape(-1,1))
    y_pred = reg.predict(np.array(x).reshape(-1,1))
    axs[dv].plot(x, y_pred,'-', color="black",linewidth=2)
    axs[dv].text(x=x.min(),y=0.18, s='r = '+ str(np.corrcoef(df_total.x,df_total.y)[0,1].round(3)))
    axs[dv].set(xlabel = name_lst[dv], ylabel = 'Eccentricity',ylim = (-0.4,0.2))

### orientation

In [ ]:
data = shapes.load_shapes("/home/yuchen/shapes/data/shapes.h5", subjects=['12-005','51-009','52-001'],stim_class=None)
data = data[data['electrode2'] == str()].reset_index(drop=True)

df_total = pd.DataFrame()
sub = ['12-005','51-009','52-001']

for subj in sub:
    df = data[['subject','amp1','freq','electrode1']].drop_duplicates()
    df = df[df.subject == subj].reset_index(drop=True)
    lst = []
    lst_area = []
    for i in range(len(df)):
        df_temp = data[(data.amp1 == df.amp1[i]) & (data.freq == df.freq[i]) & (data.subject == df.subject[i]) & (data.electrode1 == df.electrode1[i])].reset_index(drop=True)
        num = len(df_temp[df_temp.num_regions > 1]) / len(df_temp)
        lst.append(num)
        if num == 0:
            lst_area.append(df_temp.orientation.mean())

        # if there are more than one phosphene in a drawing, pick the phosphene whose size, orientation, and eccentricity is closer to the average size, orientation, and eccentricity in that group
        elif num <= 0.5:
            avg_area = df_temp[df_temp.num_regions == 1][['area']].mean().tolist()[0]
            avg_orientation = df_temp[df_temp.num_regions == 1][['orientation']].mean().tolist()[0]
            avg_eccentricity = df_temp[df_temp.num_regions == 1][['eccentricity']].mean().tolist()[0]
            for item in range(len(df_temp)):
                if df_temp.num_regions[item] > 1:
                        label_img = skimage.measure.label(df_temp['image'][item]>0)
                        regions = regionprops(label_img)
                        props = regionprops_table(label_img, properties=('centroid',
                                                                         'orientation',
                                                                         'eccentricity',
                                                                         'major_axis_length',
                                                                         'minor_axis_length',
                                                                         'area'))
                        df_similar = pd.DataFrame(props).astype('object')
                        area_id = 0
                        area_compare = 9999999
                        for j in range(len(df_similar)):
                            result_temp = (df_similar.iloc[:, 6].tolist()[j] - avg_area)**2 + (df_similar.iloc[:, 2].tolist()[j] - avg_orientation)**2 + (df_similar.iloc[:, 3].tolist()[j] - avg_eccentricity)**2
                            if result_temp < area_compare:
                                area_id = j
                                area_compare = result_temp
                        df_temp.at[item,'orientation'] = df_similar.iloc[:, 2].tolist()[area_id]
            lst_area.append(df_temp.orientation.mean())
        else:
            lst_area.append(0)

    df['helper'] = lst
    df['area'] = lst_area
    df = df[df.helper <= 0.5].reset_index(drop = True)
    df = df.drop(columns = ['helper'])
    df = df.groupby(['subject','amp1','freq','electrode1']).mean().reset_index()


    lst = []
    lst_dtf = []
    lst_d = []
    upper = []
    tan = []
    
    if subj == '12-005':
        lst_d = [0,0,0,2,7,7,0,0,5,0,
                0,0,4,11,13,15,15,3,7,4,
                0,0,15,16,17,17,19,16,9,4,
                0,0,16,19,15,17,22,25,13,10,
                0,0,8,15,14,13,17,23,14,2,
                0,0,10,15,13,12,9,11,5,-1]  # only 59 elements bc F10's distance is unidentifiable
    else:
        lst_d = [0] * 60
    s2 = shapes.subject_params[subj]
    implant,model = shapes.model_from_params(s2)
    for i in string.ascii_uppercase[0:6]: 
        for j in range(1,11):
            electrode = i + str(j)
            lst.append(electrode)
            lst_dtf.append(math.sqrt(implant[electrode].x**2 +implant[electrode].y**2 ))
            
            if implant[electrode].y > 0:
                upper.append(1)
            else:
                upper.append(0)
            e1_x = implant[electrode].x
            e1_y = implant[electrode].y
            tan.append(model.calc_bundle_tangent(e1_x,e1_y) * -1)
    df_o = pd.DataFrame(lst, columns=['electrode1'])
    df_o['distance_to_fovea'] = lst_dtf
    df_o['distance_to_implant'] = lst_d
    df_o['tan'] = tan
    
    if subj == '12-005':
        lst = lst[:-1]
    elif subj == '52-001':
        lst = ['F1','F2','F4','F5','F6','F7','F8','F9','F10',
               'E1','E2','E3','E4','E5','E6','E7','E8','E9','E10', 
               'D1','D2','D3','D4','D5','D6','D7','D8','D9','D10', 
               'C3','C4','C5','C6','C7','C8','C9','C10', 
               'B1','B3','B4','B5','B6','B7','B8','B9','B10', 
               'A1','A2','A3','A4','A5','A6','A7','A8','A9','A10']
    else:
        lst = ['F1','F2','F3','F4','F5','F6','F7','F8','F9','F10',
               'E1','E2','E3','E4','E5','E6','E7','E8','E9','E10', 
               'D1','D2','D3','D4','D5','D6','D7','D8','D9','D10', 
               'C2','C3','C4','C5','C6','C7','C8','C9','C10', 
               'B3','B4','B5','B6','B7','B8','B9','B10', 
               'A3','A4','A5','A6','A7','A8','A9','A10']
        
    df_o = df_o[df_o.electrode1.isin(lst)]
    
    df = df_o.merge(df, how = 'inner', on = 'electrode1')

    df_total = pd.concat([df_total, df])


In [ ]:
df_total = df_total.reset_index(drop=True)
for i in range(len(df_total)):
    if df_total.tan[i] < 0:
        df_total.at[i,'tan'] = df_total.tan[i]+np.pi
    if df_total.area[i] < 0:
        df_total.at[i,'area'] = df_total.area[i]+np.pi
df_AllSub = df_total.copy()
# for i in sub:
#     temp = df_total[df_total.subject == i].reset_index(drop=True)
#     temp.to_csv('/home/yuchen/shapes/notebooks/' + i + '.csv')
# df_total.to_csv('/home/yuchen/shapes/notebooks/out.csv')

In [ ]:
copy = df_total.copy()
df_s = pd.DataFrame({})
for subj in ['12-005','51-009','52-001','All']:
    if subj=='All':
        df_t = df_AllSub.copy()
    else:   
        df_t = copy[copy.subject == subj]
    df_t['amp1'] = (df_t['amp1'] - df_t['amp1'].mean()) / df_t['amp1'].std()
    df_t['freq'] = (df_t['freq'] - df_t['freq'].mean()) / df_t['freq'].std()
    df_t['distance_to_implant'] = (df_t['distance_to_implant'] - df_t['distance_to_implant'].mean()) / df_t['distance_to_implant'].std()
    df_t['distance_to_fovea'] = (df_t['distance_to_fovea'] - df_t['distance_to_fovea'].mean()) / df_t['distance_to_fovea'].std()
    df_t['tan'] = (df_t['tan'] - df_t['tan'].mean()) / df_t['tan'].std()
    if subj == 'All':  
        df_AllSub = df_t.copy()
    else:
        df_s = pd.concat([df_s, df_t])
        df_s = df_s.fillna(0)
        
df_total = df_s.copy().reset_index(drop=True)

In [ ]:
# fig, axs= plt.subplots(ncols=5, figsize=(70, 14))
# column_lst = ['amp1', 'freq','distance_to_fovea', 'distance_to_implant','tan']
# name_lst = ['Amplitude','Frequency','Foveal Eccentricity', 'Electrode-Implant Distance','Axonal Tangential Line']
# for dv in range(5):
#     reg = LinearRegression().fit(df_total[column_lst[:dv] + column_lst[dv+1 :]], df_total[['area']])
#     y_predicted = reg.predict(df_total[column_lst[:dv] + column_lst[dv+1 :]])
#     y = df_total[['area']]-y_predicted

#     reg = LinearRegression().fit(df_total[column_lst[:dv] + column_lst[dv+1 :]], df_total[column_lst[dv]])
#     y_predicted = reg.predict(df_total[column_lst[:dv] + column_lst[dv+1 :]])
#     x = df_total[column_lst[dv]]-y_predicted
#     df_total['x'] = x
#     df_total['y'] = y
#     subject = ['12-005','51-009','52-001']
#     marker_lst = ['o','s','^']
#     color_lst = ['#1E88E5', '#FFC107','#004D40']
#     for i in range(len(subject)):
#         temp = df_total[df_total.subject == subject[i]]
#         axs[dv].plot(temp['x'],temp['y'],marker_lst[i], color=color_lst[i],alpha=0.6,markersize=20)
#     axs[dv].legend(subject, loc='upper right',prop={'size': 30})
#     reg = LinearRegression().fit(np.array(x).reshape(-1,1),np.array(y).reshape(-1,1))
#     y_pred = reg.predict(np.array(x).reshape(-1,1))
#     axs[dv].plot(x, y_pred,'-', color="black",linewidth=2)
#     axs[dv].set(xlabel = name_lst[dv], ylabel = 'Orientation',ylim=(-2.5,3.5))
#     for item in ([axs[dv].xaxis.label, axs[dv].yaxis.label]+axs[dv].get_xticklabels()+axs[dv].get_yticklabels()):
#         item.set_fontsize(30)
# fig.savefig('/home/yuchen/paper/11b. Single-Electrode Orientation.pdf', transparent=True)

In [ ]:
for subj in ['12-005','51-009','52-001','AllSubjects']:
    temp = df_total[df_total.subject == subj].reset_index(drop=True)
    if subj == '12-005':
        X = temp[['amp1','freq','distance_to_fovea','distance_to_implant','tan']]
    elif subj == 'AllSubjects':
        temp = df_AllSub.copy()
        X = temp[['amp1','freq','distance_to_fovea','distance_to_implant','tan']]
    else:
        X = temp[['amp1','freq','distance_to_fovea','tan']]
    y = temp.area
    X2 = sm.add_constant(X)
    est = sm.OLS(y, X2)
    est2 = est.fit()
    print('subject: ' + subj)
    print(est2.summary())
    print('\ncoefficient with more digits: ')
    print(est2.params)
    print('\nvariance inflation factor: ')
    print([[X.columns[i],variance_inflation_factor(X.values, i)]
                          for i in range(len(X.columns))])
    print('\npartial correlation: ')
    print(temp[['amp1','freq','distance_to_fovea','distance_to_implant','tan','area']].pcorr())
    print(' ')

In [ ]:
est2.summary().as_csv('est2_summary.txt', sep='\t')

### major axis

In [ ]:
data = shapes.load_shapes("/home/yuchen/shapes/data/shapes.h5", subjects=['12-005','51-009','52-001'],stim_class=None)
data = data[data['electrode2'] == str()].reset_index(drop=True)

df_total = pd.DataFrame()
df_AllSub = pd.DataFrame()
sub = ['12-005','51-009','52-001']

for subj in sub:
    df = data[['subject','amp1','freq','electrode1']].drop_duplicates().reset_index(drop=True)
    df = df[df.subject == subj].reset_index(drop=True)
    lst_major = []
    lst_minor = []
    lst_perimeter  = []
    for i in range(len(data)):
        label_img = skimage.measure.label(data['image'][i]>0)
        regions = regionprops(label_img)
        props = regionprops_table(label_img, properties=('centroid',
                                    'orientation',
                                    'eccentricity',
                                    'major_axis_length',
                                    'minor_axis_length',
                                    'area',
                                    'perimeter'))
        df_similar = pd.DataFrame(props).astype('object')
        lst_major.append(df_similar['major_axis_length'][0])
        lst_minor.append(df_similar['minor_axis_length'][0])
        lst_perimeter.append(df_similar['perimeter'][0])
    data['major_axis_length'] = lst_major
    data['minor_axis_length'] = lst_minor
    data['perimeter'] = lst_perimeter

    lst = []
    lst_area = []
    for i in range(len(df)):
        df_temp = data[(data.amp1 == df.amp1[i]) & (data.freq == df.freq[i]) & (data.subject == df.subject[i]) & (data.electrode1 == df.electrode1[i])].reset_index(drop=True)
        num = len(df_temp[df_temp.num_regions > 1]) / len(df_temp)
        lst.append(num)
        if num == 0:
            lst_area.append(df_temp.major_axis_length.mean())

        # if there are more than one phosphene in a drawing, pick the phosphene whose size, orientation, and eccentricity is closer to the average size, orientation, and eccentricity in that group
        elif num <= 0.5:
            avg_area = df_temp[df_temp.num_regions == 1][['area']].mean().tolist()[0]
            avg_orientation = df_temp[df_temp.num_regions == 1][['orientation']].mean().tolist()[0]
            avg_eccentricity = df_temp[df_temp.num_regions == 1][['eccentricity']].mean().tolist()[0]
            for item in range(len(df_temp)):
                if df_temp.num_regions[item] > 1:
                        label_img = skimage.measure.label(df_temp['image'][item]>0)
                        regions = regionprops(label_img)
                        props = regionprops_table(label_img, properties=('centroid',
                                                                         'orientation',
                                                                         'eccentricity',
                                                                         'major_axis_length',
                                                                         'minor_axis_length',
                                                                         'area',
                                                                        'perimeter'))
                        df_similar = pd.DataFrame(props).astype('object')
                        area_id = 0
                        area_compare = 9999999
                        for j in range(len(df_similar)):
                            result_temp = (df_similar.iloc[:, 6].tolist()[j] - avg_area)**2 + (df_similar.iloc[:, 2].tolist()[j] - avg_orientation)**2 + (df_similar.iloc[:, 3].tolist()[j] - avg_eccentricity)**2
                            if result_temp < area_compare:
                                area_id = j
                                area_compare = result_temp
                        df_temp.at[item,'major_axis_length'] = df_similar.iloc[:, 4].tolist()[area_id]
            lst_area.append(df_temp.major_axis_length.mean())
        else:
            lst_area.append(0)

    df['helper'] = lst
    df['area'] = lst_area
    df = df[df.helper <= 0.5].reset_index(drop = True)
    df = df.drop(columns = ['helper'])
    df = df.groupby(['subject','amp1','freq','electrode1']).mean().reset_index()


    lst = []
    lst_dtf = []
    lst_d = []
    upper = []
    
    if subj == '12-005':
        lst_d = [0,0,0,2,7,7,0,0,5,0,
                0,0,4,11,13,15,15,3,7,4,
                0,0,15,16,17,17,19,16,9,4,
                0,0,16,19,15,17,22,25,13,10,
                0,0,8,15,14,13,17,23,14,2,
                0,0,10,15,13,12,9,11,5,-1]  # only 59 elements bc F10's distance is unidentifiable
    else:
        lst_d = [0] * 60
    s2 = shapes.subject_params[subj]
    implant,model = shapes.model_from_params(s2)
    for i in string.ascii_uppercase[0:6]: 
        for j in range(1,11):
            electrode = i + str(j)
            lst.append(electrode)
            lst_dtf.append(math.sqrt(implant[electrode].x**2 +implant[electrode].y**2 ))
            
            if implant[electrode].y > 0:
                upper.append(1)
            else:
                upper.append(0)
            e1_x = implant[electrode].x
            e1_y = implant[electrode].y

    df_o = pd.DataFrame(lst, columns=['electrode1'])
    df_o['distance_to_fovea'] = lst_dtf
    df_o['distance_to_implant'] = lst_d
    
    if subj == '12-005':
        lst = lst[:-1]
    elif subj == '52-001':
        lst = ['F1','F2','F4','F5','F6','F7','F8','F9','F10',
               'E1','E2','E3','E4','E5','E6','E7','E8','E9','E10', 
               'D1','D2','D3','D4','D5','D6','D7','D8','D9','D10', 
               'C3','C4','C5','C6','C7','C8','C9','C10', 
               'B1','B3','B4','B5','B6','B7','B8','B9','B10', 
               'A1','A2','A3','A4','A5','A6','A7','A8','A9','A10']
    else:
        lst = ['F1','F2','F3','F4','F5','F6','F7','F8','F9','F10',
               'E1','E2','E3','E4','E5','E6','E7','E8','E9','E10', 
               'D1','D2','D3','D4','D5','D6','D7','D8','D9','D10', 
               'C2','C3','C4','C5','C6','C7','C8','C9','C10', 
               'B3','B4','B5','B6','B7','B8','B9','B10', 
               'A3','A4','A5','A6','A7','A8','A9','A10']
        
    df_o = df_o[df_o.electrode1.isin(lst)]
    
    df = df_o.merge(df, how = 'inner', on = 'electrode1')
    
    df_reference = df[(df.subject == subj) & (df.amp1 == 2) & (df.freq == 20) & 
                      (df.distance_to_implant == 0) & 
                      (df.distance_to_fovea>2950) & (df.distance_to_fovea<3450)].reset_index(drop=True)
    reference_number  = df_reference.area.mean()
    for i in range(len(df)):
        df.at[i, 'area'] = df.at[i, 'area'] / reference_number
        
    df_total = pd.concat([df_total, df])

for i in sub:
    temp = df_total[df_total.subject == i].reset_index(drop=True)
    temp.to_csv('/home/yuchen/shapes/notebooks/' + i + '.csv')
df_total.to_csv('/home/yuchen/shapes/notebooks/out.csv')

In [ ]:
copy = df_total.copy()
df_AllSub = df_total.copy()
df_s = pd.DataFrame({})
for subj in ['12-005','51-009','52-001','All']:
    if subj=='All':
        df_t = df_AllSub.copy()
    else:   
        df_t = copy[copy.subject == subj]
    df_t['amp1'] = (df_t['amp1'] - df_t['amp1'].mean()) / df_t['amp1'].std()
    df_t['freq'] = (df_t['freq'] - df_t['freq'].mean()) / df_t['freq'].std()
    df_t['distance_to_implant'] = (df_t['distance_to_implant'] - df_t['distance_to_implant'].mean()) / df_t['distance_to_implant'].std()
    df_t['distance_to_fovea'] = (df_t['distance_to_fovea'] - df_t['distance_to_fovea'].mean()) / df_t['distance_to_fovea'].std()
    if subj == 'All':  
        df_AllSub = df_t.copy()
    else:
        df_s = pd.concat([df_s, df_t])
        df_s = df_s.fillna(0)
        
df_total = df_s.copy().reset_index(drop=True)

In [ ]:
fig, axs= plt.subplots(ncols=4, figsize=(56, 14))
column_lst = ['amp1', 'freq','distance_to_fovea', 'distance_to_implant']
name_lst = ['Amplitude','Frequency','Foveal Eccentricity', 'Electrode-Implant Distance']
for dv in range(4):
    reg = LinearRegression().fit(df_total[column_lst[:dv] + column_lst[dv+1 :]], df_total[['area']])
    y_predicted = reg.predict(df_total[column_lst[:dv] + column_lst[dv+1 :]])
    y = df_total[['area']]-y_predicted

    reg = LinearRegression().fit(df_total[column_lst[:dv] + column_lst[dv+1 :]], df_total[column_lst[dv]])
    y_predicted = reg.predict(df_total[column_lst[:dv] + column_lst[dv+1 :]])
    x = df_total[column_lst[dv]]-y_predicted
    df_total['x'] = x
    df_total['y'] = y
    subject = ['12-005','51-009','52-001']
    marker_lst = ['o','s','^']
    color_lst = ['#1E88E5', '#FFC107','#004D40']
    for i in range(len(subject)):
        temp = df_total[df_total.subject == subject[i]]
        axs[dv].plot(temp['x'],temp['y'],marker_lst[i], color=color_lst[i],alpha=0.6,markersize=20)
    axs[dv].legend(subject, loc='upper right',prop={'size': 30})
    reg = LinearRegression().fit(np.array(x).reshape(-1,1),np.array(y).reshape(-1,1))
    y_pred = reg.predict(np.array(x).reshape(-1,1))
    axs[dv].plot(x, y_pred,'-', color="black",linewidth=2)
    axs[dv].set(xlabel = name_lst[dv], ylabel = 'Major Axis Length',ylim=(-2.5,4.5))
    for item in ([axs[dv].xaxis.label, axs[dv].yaxis.label]+axs[dv].get_xticklabels()+axs[dv].get_yticklabels()):
        item.set_fontsize(30)
fig.savefig('/home/yuchen/paper/11c. Single-Electrode Major Axis Length.pdf', transparent=True)

In [ ]:
for subj in ['12-005','51-009','52-001','AllSubjects']:
    temp = df_total[df_total.subject == subj].reset_index(drop=True)
    if subj == '12-005':
        X = temp[['amp1','freq','distance_to_fovea','distance_to_implant']]
    elif subj == 'AllSubjects':
        temp = df_AllSub.copy()
        X = temp[['amp1','freq','distance_to_fovea','distance_to_implant']]
    else:
        X = temp[['amp1','freq','distance_to_fovea']]
    y = temp.area
    X2 = sm.add_constant(X)
    est = sm.OLS(y, X2)
    est2 = est.fit()
    print('subject: ' + subj)
    print(est2.summary())
    print('\ncoefficient with more digits: ')
    print(est2.params)
    print('\nvariance inflation factor: ')
    print([[X.columns[i],variance_inflation_factor(X.values, i)]
                          for i in range(len(X.columns))])
    print('\npartial correlation: ')
    X['area'] = temp['area']
    print(X.pcorr())
    print(' ')

### minor axis

In [ ]:
data = shapes.load_shapes("/home/yuchen/shapes/data/shapes.h5", subjects=['12-005','51-009','52-001'],stim_class=None)
data = data[data['electrode2'] == str()].reset_index(drop=True)

df_total = pd.DataFrame()
df_AllSub = pd.DataFrame()
sub = ['12-005','51-009','52-001']

for subj in sub:
    df = data[['subject','amp1','freq','electrode1']].drop_duplicates().reset_index(drop=True)
    df = df[df.subject == subj].reset_index(drop=True)
    lst_major = []
    lst_minor = []
    lst_perimeter  = []
    for i in range(len(data)):
        label_img = skimage.measure.label(data['image'][i]>0)
        regions = regionprops(label_img)
        props = regionprops_table(label_img, properties=('centroid',
                                    'orientation',
                                    'eccentricity',
                                    'major_axis_length',
                                    'minor_axis_length',
                                    'area',
                                    'perimeter'))
        df_similar = pd.DataFrame(props).astype('object')
        lst_major.append(df_similar['major_axis_length'][0])
        lst_minor.append(df_similar['minor_axis_length'][0])
        lst_perimeter.append(df_similar['perimeter'][0])
    data['major_axis_length'] = lst_major
    data['minor_axis_length'] = lst_minor
    data['perimeter'] = lst_perimeter

    lst = []
    lst_area = []
    for i in range(len(df)):
        df_temp = data[(data.amp1 == df.amp1[i]) & (data.freq == df.freq[i]) & (data.subject == df.subject[i]) & (data.electrode1 == df.electrode1[i])].reset_index(drop=True)
        num = len(df_temp[df_temp.num_regions > 1]) / len(df_temp)
        lst.append(num)
        if num == 0:
            lst_area.append(df_temp.minor_axis_length.mean())

        # if there are more than one phosphene in a drawing, pick the phosphene whose size, orientation, and eccentricity is closer to the average size, orientation, and eccentricity in that group
        elif num <= 0.5:
            avg_area = df_temp[df_temp.num_regions == 1][['area']].mean().tolist()[0]
            avg_orientation = df_temp[df_temp.num_regions == 1][['orientation']].mean().tolist()[0]
            avg_eccentricity = df_temp[df_temp.num_regions == 1][['eccentricity']].mean().tolist()[0]
            for item in range(len(df_temp)):
                if df_temp.num_regions[item] > 1:
                        label_img = skimage.measure.label(df_temp['image'][item]>0)
                        regions = regionprops(label_img)
                        props = regionprops_table(label_img, properties=('centroid',
                                                                         'orientation',
                                                                         'eccentricity',
                                                                         'major_axis_length',
                                                                         'minor_axis_length',
                                                                         'area',
                                                                        'perimeter'))
                        df_similar = pd.DataFrame(props).astype('object')
                        area_id = 0
                        area_compare = 9999999
                        for j in range(len(df_similar)):
                            result_temp = (df_similar.iloc[:, 6].tolist()[j] - avg_area)**2 + (df_similar.iloc[:, 2].tolist()[j] - avg_orientation)**2 + (df_similar.iloc[:, 3].tolist()[j] - avg_eccentricity)**2
                            if result_temp < area_compare:
                                area_id = j
                                area_compare = result_temp
                        df_temp.at[item,'minor_axis_length'] = df_similar.iloc[:, 5].tolist()[area_id]
            lst_area.append(df_temp.minor_axis_length.mean())
        else:
            lst_area.append(0)

    df['helper'] = lst
    df['area'] = lst_area
    df = df[df.helper <= 0.5].reset_index(drop = True)
    df = df.drop(columns = ['helper'])
    df = df.groupby(['subject','amp1','freq','electrode1']).mean().reset_index()


    lst = []
    lst_dtf = []
    lst_d = []
    upper = []
    
    if subj == '12-005':
        lst_d = [0,0,0,2,7,7,0,0,5,0,
                0,0,4,11,13,15,15,3,7,4,
                0,0,15,16,17,17,19,16,9,4,
                0,0,16,19,15,17,22,25,13,10,
                0,0,8,15,14,13,17,23,14,2,
                0,0,10,15,13,12,9,11,5,-1]  # only 59 elements bc F10's distance is unidentifiable
    else:
        lst_d = [0] * 60
    s2 = shapes.subject_params[subj]
    implant,model = shapes.model_from_params(s2)
    for i in string.ascii_uppercase[0:6]: 
        for j in range(1,11):
            electrode = i + str(j)
            lst.append(electrode)
            lst_dtf.append(math.sqrt(implant[electrode].x**2 +implant[electrode].y**2 ))
            
            if implant[electrode].y > 0:
                upper.append(1)
            else:
                upper.append(0)
            e1_x = implant[electrode].x
            e1_y = implant[electrode].y

    df_o = pd.DataFrame(lst, columns=['electrode1'])
    df_o['distance_to_fovea'] = lst_dtf
    df_o['distance_to_implant'] = lst_d
    
    if subj == '12-005':
        lst = lst[:-1]
    elif subj == '52-001':
        lst = ['F1','F2','F4','F5','F6','F7','F8','F9','F10',
               'E1','E2','E3','E4','E5','E6','E7','E8','E9','E10', 
               'D1','D2','D3','D4','D5','D6','D7','D8','D9','D10', 
               'C3','C4','C5','C6','C7','C8','C9','C10', 
               'B1','B3','B4','B5','B6','B7','B8','B9','B10', 
               'A1','A2','A3','A4','A5','A6','A7','A8','A9','A10']
    else:
        lst = ['F1','F2','F3','F4','F5','F6','F7','F8','F9','F10',
               'E1','E2','E3','E4','E5','E6','E7','E8','E9','E10', 
               'D1','D2','D3','D4','D5','D6','D7','D8','D9','D10', 
               'C2','C3','C4','C5','C6','C7','C8','C9','C10', 
               'B3','B4','B5','B6','B7','B8','B9','B10', 
               'A3','A4','A5','A6','A7','A8','A9','A10']
        
    df_o = df_o[df_o.electrode1.isin(lst)]
    
    df = df_o.merge(df, how = 'inner', on = 'electrode1')
    
    df_reference = df[(df.subject == subj) & (df.amp1 == 2) & (df.freq == 20) & 
                      (df.distance_to_implant == 0) & 
                      (df.distance_to_fovea>2950) & (df.distance_to_fovea<3450)].reset_index(drop=True)
    reference_number  = df_reference.area.mean()
    for i in range(len(df)):
        df.at[i, 'area'] = df.at[i, 'area'] / reference_number
        
    df_total = pd.concat([df_total, df])

for i in sub:
    temp = df_total[df_total.subject == i].reset_index(drop=True)
    temp.to_csv('/home/yuchen/shapes/notebooks/' + i + '.csv')
df_total.to_csv('/home/yuchen/shapes/notebooks/out.csv')

In [ ]:
copy = df_total.copy()
df_AllSub = df_total.copy()
df_s = pd.DataFrame({})
for subj in ['12-005','51-009','52-001','All']:
    if subj=='All':
        df_t = df_AllSub.copy()
    else:   
        df_t = copy[copy.subject == subj]
    df_t['amp1'] = (df_t['amp1'] - df_t['amp1'].mean()) / df_t['amp1'].std()
    df_t['freq'] = (df_t['freq'] - df_t['freq'].mean()) / df_t['freq'].std()
    df_t['distance_to_implant'] = (df_t['distance_to_implant'] - df_t['distance_to_implant'].mean()) / df_t['distance_to_implant'].std()
    df_t['distance_to_fovea'] = (df_t['distance_to_fovea'] - df_t['distance_to_fovea'].mean()) / df_t['distance_to_fovea'].std()
    if subj == 'All':  
        df_AllSub = df_t.copy()
    else:
        df_s = pd.concat([df_s, df_t])
        df_s = df_s.fillna(0)
        
df_total = df_s.copy().reset_index(drop=True)

In [ ]:
fig, axs= plt.subplots(ncols=4, figsize=(56,14))
column_lst = ['amp1', 'freq','distance_to_fovea', 'distance_to_implant']
name_lst = ['Amplitude','Frequency','Foveal Eccentricity', 'Electrode-Implant Distance']
for dv in range(4):
    reg = LinearRegression().fit(df_total[column_lst[:dv] + column_lst[dv+1 :]], df_total[['area']])
    y_predicted = reg.predict(df_total[column_lst[:dv] + column_lst[dv+1 :]])
    y = df_total[['area']]-y_predicted

    reg = LinearRegression().fit(df_total[column_lst[:dv] + column_lst[dv+1 :]], df_total[column_lst[dv]])
    y_predicted = reg.predict(df_total[column_lst[:dv] + column_lst[dv+1 :]])
    x = df_total[column_lst[dv]]-y_predicted
    df_total['x'] = x
    df_total['y'] = y
    subject = ['12-005','51-009','52-001']
    marker_lst = ['o','s','^']
    color_lst = ['#1E88E5', '#FFC107','#004D40']
    for i in range(len(subject)):
        temp = df_total[df_total.subject == subject[i]]
        axs[dv].plot(temp['x'],temp['y'],marker_lst[i], color=color_lst[i],alpha=0.6,markersize=20)
    axs[dv].legend(subject, loc='upper right',prop={'size': 30})
    reg = LinearRegression().fit(np.array(x).reshape(-1,1),np.array(y).reshape(-1,1))
    y_pred = reg.predict(np.array(x).reshape(-1,1))
    axs[dv].plot(x, y_pred,'-', color="black",linewidth=2)
    axs[dv].set(xlabel = name_lst[dv], ylabel = 'Minor Axis Length',ylim=(-2,3.5))
    for item in ([axs[dv].xaxis.label, axs[dv].yaxis.label]+axs[dv].get_xticklabels()+axs[dv].get_yticklabels()):
        item.set_fontsize(30)
fig.savefig('/home/yuchen/paper/11d. Single-Electrode Minor Axis Length.pdf', transparent=True)

In [ ]:
for subj in ['12-005','51-009','52-001','AllSubjects']:
    temp = df_total[df_total.subject == subj].reset_index(drop=True)
    if subj == '12-005':
        X = temp[['amp1','freq','distance_to_fovea','distance_to_implant']]
    elif subj == 'AllSubjects':
        temp = df_AllSub.copy()
        X = temp[['amp1','freq','distance_to_fovea','distance_to_implant']]
    else:
        X = temp[['amp1','freq','distance_to_fovea']]
    y = temp.area
    X2 = sm.add_constant(X)
    est = sm.OLS(y, X2)
    est2 = est.fit()
    print('subject: ' + subj)
    print(est2.summary())
    print('\ncoefficient with more digits: ')
    print(est2.params)
    print('\nvariance inflation factor: ')
    print([[X.columns[i],variance_inflation_factor(X.values, i)]
                          for i in range(len(X.columns))])
    print('\npartial correlation: ')
    X['area'] = temp['area']
    print(X.pcorr())
    print(' ')

### perimeter

In [ ]:
data = shapes.load_shapes("/home/yuchen/shapes/data/shapes.h5", subjects=['12-005','51-009','52-001'],stim_class=None)
data = data[data['electrode2'] == str()].reset_index(drop=True)

df_total = pd.DataFrame()
df_AllSub = pd.DataFrame()
sub = ['12-005','51-009','52-001']

for subj in sub:
    df = data[['subject','amp1','freq','electrode1']].drop_duplicates().reset_index(drop=True)
    df = df[df.subject == subj].reset_index(drop=True)
    lst_major = []
    lst_minor = []
    lst_perimeter  = []
    for i in range(len(data)):
        label_img = skimage.measure.label(data['image'][i]>0)
        regions = regionprops(label_img)
        props = regionprops_table(label_img, properties=('centroid',
                                    'orientation',
                                    'eccentricity',
                                    'major_axis_length',
                                    'minor_axis_length',
                                    'area',
                                    'perimeter'))
        df_similar = pd.DataFrame(props).astype('object')
        lst_major.append(df_similar['major_axis_length'][0])
        lst_minor.append(df_similar['minor_axis_length'][0])
        lst_perimeter.append(df_similar['perimeter'][0])
    data['major_axis_length'] = lst_major
    data['minor_axis_length'] = lst_minor
    data['perimeter'] = lst_perimeter

    lst = []
    lst_area = []
    for i in range(len(df)):
        df_temp = data[(data.amp1 == df.amp1[i]) & (data.freq == df.freq[i]) & (data.subject == df.subject[i]) & (data.electrode1 == df.electrode1[i])].reset_index(drop=True)
        num = len(df_temp[df_temp.num_regions > 1]) / len(df_temp)
        lst.append(num)
        if num == 0:
            lst_area.append(df_temp.perimeter.mean())

        # if there are more than one phosphene in a drawing, pick the phosphene whose size, orientation, and eccentricity is closer to the average size, orientation, and eccentricity in that group
        elif num <= 0.5:
            avg_area = df_temp[df_temp.num_regions == 1][['area']].mean().tolist()[0]
            avg_orientation = df_temp[df_temp.num_regions == 1][['orientation']].mean().tolist()[0]
            avg_eccentricity = df_temp[df_temp.num_regions == 1][['eccentricity']].mean().tolist()[0]
            for item in range(len(df_temp)):
                if df_temp.num_regions[item] > 1:
                        label_img = skimage.measure.label(df_temp['image'][item]>0)
                        regions = regionprops(label_img)
                        props = regionprops_table(label_img, properties=('centroid',
                                                                         'orientation',
                                                                         'eccentricity',
                                                                         'major_axis_length',
                                                                         'minor_axis_length',
                                                                         'area',
                                                                        'perimeter'))
                        df_similar = pd.DataFrame(props).astype('object')
                        area_id = 0
                        area_compare = 9999999
                        for j in range(len(df_similar)):
                            result_temp = (df_similar.iloc[:, 6].tolist()[j] - avg_area)**2 + (df_similar.iloc[:, 2].tolist()[j] - avg_orientation)**2 + (df_similar.iloc[:, 3].tolist()[j] - avg_eccentricity)**2
                            if result_temp < area_compare:
                                area_id = j
                                area_compare = result_temp
                        df_temp.at[item,'perimeter'] = df_similar.iloc[:, 7].tolist()[area_id]
            lst_area.append(df_temp.perimeter.mean())
        else:
            lst_area.append(0)

    df['helper'] = lst
    df['area'] = lst_area
    df = df[df.helper <= 0.5].reset_index(drop = True)
    df = df.drop(columns = ['helper'])
    df = df.groupby(['subject','amp1','freq','electrode1']).mean().reset_index()


    lst = []
    lst_dtf = []
    lst_d = []
    upper = []
    
    if subj == '12-005':
        lst_d = [0,0,0,2,7,7,0,0,5,0,
                0,0,4,11,13,15,15,3,7,4,
                0,0,15,16,17,17,19,16,9,4,
                0,0,16,19,15,17,22,25,13,10,
                0,0,8,15,14,13,17,23,14,2,
                0,0,10,15,13,12,9,11,5,-1]  # only 59 elements bc F10's distance is unidentifiable
    else:
        lst_d = [0] * 60
    s2 = shapes.subject_params[subj]
    implant,model = shapes.model_from_params(s2)
    for i in string.ascii_uppercase[0:6]: 
        for j in range(1,11):
            electrode = i + str(j)
            lst.append(electrode)
            lst_dtf.append(math.sqrt(implant[electrode].x**2 +implant[electrode].y**2 ))
            
            if implant[electrode].y > 0:
                upper.append(1)
            else:
                upper.append(0)
            e1_x = implant[electrode].x
            e1_y = implant[electrode].y

    df_o = pd.DataFrame(lst, columns=['electrode1'])
    df_o['distance_to_fovea'] = lst_dtf
    df_o['distance_to_implant'] = lst_d
    
    if subj == '12-005':
        lst = lst[:-1]
    elif subj == '52-001':
        lst = ['F1','F2','F4','F5','F6','F7','F8','F9','F10',
               'E1','E2','E3','E4','E5','E6','E7','E8','E9','E10', 
               'D1','D2','D3','D4','D5','D6','D7','D8','D9','D10', 
               'C3','C4','C5','C6','C7','C8','C9','C10', 
               'B1','B3','B4','B5','B6','B7','B8','B9','B10', 
               'A1','A2','A3','A4','A5','A6','A7','A8','A9','A10']
    else:
        lst = ['F1','F2','F3','F4','F5','F6','F7','F8','F9','F10',
               'E1','E2','E3','E4','E5','E6','E7','E8','E9','E10', 
               'D1','D2','D3','D4','D5','D6','D7','D8','D9','D10', 
               'C2','C3','C4','C5','C6','C7','C8','C9','C10', 
               'B3','B4','B5','B6','B7','B8','B9','B10', 
               'A3','A4','A5','A6','A7','A8','A9','A10']
        
    df_o = df_o[df_o.electrode1.isin(lst)]
    
    df = df_o.merge(df, how = 'inner', on = 'electrode1')
    
    df_reference = df[(df.subject == subj) & (df.amp1 == 2) & (df.freq == 20) & 
                      (df.distance_to_implant == 0) & 
                      (df.distance_to_fovea>2950) & (df.distance_to_fovea<3450)].reset_index(drop=True)
    reference_number  = df_reference.area.mean()
    for i in range(len(df)):
        df.at[i, 'area'] = df.at[i, 'area'] / reference_number
        
    df_total = pd.concat([df_total, df])

for i in sub:
    temp = df_total[df_total.subject == i].reset_index(drop=True)
    temp.to_csv('/home/yuchen/shapes/notebooks/' + i + '.csv')
df_total.to_csv('/home/yuchen/shapes/notebooks/out.csv')

In [ ]:
copy = df_total.copy()
df_AllSub = df_total.copy()
df_s = pd.DataFrame({})
for subj in ['12-005','51-009','52-001','All']:
    if subj=='All':
        df_t = df_AllSub.copy()
    else:   
        df_t = copy[copy.subject == subj]
    df_t['amp1'] = (df_t['amp1'] - df_t['amp1'].mean()) / df_t['amp1'].std()
    df_t['freq'] = (df_t['freq'] - df_t['freq'].mean()) / df_t['freq'].std()
    df_t['distance_to_implant'] = (df_t['distance_to_implant'] - df_t['distance_to_implant'].mean()) / df_t['distance_to_implant'].std()
    df_t['distance_to_fovea'] = (df_t['distance_to_fovea'] - df_t['distance_to_fovea'].mean()) / df_t['distance_to_fovea'].std()
    if subj == 'All':  
        df_AllSub = df_t.copy()
    else:
        df_s = pd.concat([df_s, df_t])
        df_s = df_s.fillna(0)
        
df_total = df_s.copy().reset_index(drop=True)

In [ ]:
fig, axs= plt.subplots(ncols=4, figsize=(56, 14))
column_lst = ['amp1', 'freq','distance_to_fovea', 'distance_to_implant']
name_lst = ['Amplitude','Frequency','Foveal Eccentricity', 'Electrode-Implant Distance']
for dv in range(4):
    reg = LinearRegression().fit(df_total[column_lst[:dv] + column_lst[dv+1 :]], df_total[['area']])
    y_predicted = reg.predict(df_total[column_lst[:dv] + column_lst[dv+1 :]])
    y = df_total[['area']]-y_predicted

    reg = LinearRegression().fit(df_total[column_lst[:dv] + column_lst[dv+1 :]], df_total[column_lst[dv]])
    y_predicted = reg.predict(df_total[column_lst[:dv] + column_lst[dv+1 :]])
    x = df_total[column_lst[dv]]-y_predicted
    df_total['x'] = x
    df_total['y'] = y
    subject = ['12-005','51-009','52-001']
    marker_lst = ['o','s','^']
    color_lst = ['#1E88E5', '#FFC107','#004D40']
    for i in range(len(subject)):
        temp = df_total[df_total.subject == subject[i]]
        axs[dv].plot(temp['x'],temp['y'],marker_lst[i], color=color_lst[i],alpha=0.6,markersize=20)
    axs[dv].legend(subject, loc='upper right',prop={'size': 30})
    reg = LinearRegression().fit(np.array(x).reshape(-1,1),np.array(y).reshape(-1,1))
    y_pred = reg.predict(np.array(x).reshape(-1,1))
    axs[dv].plot(x, y_pred,'-', color="black",linewidth=2)
    axs[dv].set(xlabel = name_lst[dv], ylabel = 'Perimeter',ylim=(-2.5,4.5))
    for item in ([axs[dv].xaxis.label, axs[dv].yaxis.label]+axs[dv].get_xticklabels()+axs[dv].get_yticklabels()):
        item.set_fontsize(30)
fig.savefig('/home/yuchen/paper/11e. Single-Electrode Perimeter.pdf', transparent=True)

In [ ]:
for subj in ['12-005','51-009','52-001','AllSubjects']:
    temp = df_total[df_total.subject == subj].reset_index(drop=True)
    if subj == '12-005':
        X = temp[['amp1','freq','distance_to_fovea','distance_to_implant']]
    elif subj == 'AllSubjects':
        temp = df_AllSub.copy()
        X = temp[['amp1','freq','distance_to_fovea','distance_to_implant']]
    else:
        X = temp[['amp1','freq','distance_to_fovea']]
    y = temp.area
    X2 = sm.add_constant(X)
    est = sm.OLS(y, X2)
    est2 = est.fit()
    print('subject: ' + subj)
    print(est2.summary())
    print('\ncoefficient with more digits: ')
    print(est2.params)
    print('\nvariance inflation factor: ')
    print([[X.columns[i],variance_inflation_factor(X.values, i)]
                          for i in range(len(X.columns))])
    print('\npartial correlation: ')
    X['area'] = temp['area']
    print(X.pcorr())
    print(' ')

### phosphene number

In [ ]:
data = shapes.load_shapes("/home/yuchen/shapes/data/shapes.h5", subjects=['12-005','51-009','52-001'],stim_class=None)
data = data[data['electrode2'] == str()].reset_index(drop=True)

df_total = pd.DataFrame()
sub = ['12-005','51-009','52-001']

for subj in sub:
    df = data[['subject','amp1','freq','electrode1']].drop_duplicates().reset_index(drop=True)
    df = df[df.subject == subj].reset_index(drop=True)
    lst = []
    lst_area = []
    for i in range(len(df)):
        df_temp = data[(data.amp1 == df.amp1[i]) & (data.freq == df.freq[i]) & (data.subject == df.subject[i]) & (data.electrode1 == df.electrode1[i])].reset_index(drop=True)
        num = len(df_temp[df_temp.num_regions > 1]) / len(df_temp)
        lst.append(mean(df_temp.num_regions))


    df['area'] = lst

    lst = []
    lst_dtf = []
    lst_d = []
    upper = []
    
    if subj == '12-005':
        lst_d = [0,0,0,2,7,7,0,0,5,0,
                0,0,4,11,13,15,15,3,7,4,
                0,0,15,16,17,17,19,16,9,4,
                0,0,16,19,15,17,22,25,13,10,
                0,0,8,15,14,13,17,23,14,2,
                0,0,10,15,13,12,9,11,5,-1]  # only 59 elements bc F10's distance is unidentifiable
    else:
        lst_d = [0] * 60
    s2 = shapes.subject_params[subj]
    implant,model = shapes.model_from_params(s2)
    for i in string.ascii_uppercase[0:6]: 
        for j in range(1,11):
            electrode = i + str(j)
            lst.append(electrode)
            lst_dtf.append(math.sqrt(implant[electrode].x**2 +implant[electrode].y**2 ))
            
            if implant[electrode].y > 0:
                upper.append(1)
            else:
                upper.append(0)
            e1_x = implant[electrode].x
            e1_y = implant[electrode].y

    df_o = pd.DataFrame(lst, columns=['electrode1'])
    df_o['distance_to_fovea'] = lst_dtf
    df_o['distance_to_implant'] = lst_d
    
    if subj == '12-005':
        lst = lst[:-1]
    elif subj == '52-001':
        lst = ['F1','F2','F4','F5','F6','F7','F8','F9','F10',
               'E1','E2','E3','E4','E5','E6','E7','E8','E9','E10', 
               'D1','D2','D3','D4','D5','D6','D7','D8','D9','D10', 
               'C3','C4','C5','C6','C7','C8','C9','C10', 
               'B1','B3','B4','B5','B6','B7','B8','B9','B10', 
               'A1','A2','A3','A4','A5','A6','A7','A8','A9','A10']
    else:
        lst = ['F1','F2','F3','F4','F5','F6','F7','F8','F9','F10',
               'E1','E2','E3','E4','E5','E6','E7','E8','E9','E10', 
               'D1','D2','D3','D4','D5','D6','D7','D8','D9','D10', 
               'C2','C3','C4','C5','C6','C7','C8','C9','C10', 
               'B3','B4','B5','B6','B7','B8','B9','B10', 
               'A3','A4','A5','A6','A7','A8','A9','A10']
        
    df_o = df_o[df_o.electrode1.isin(lst)]
    
    df = df_o.merge(df, how = 'inner', on = 'electrode1')

    df_total = pd.concat([df_total, df])
    
    

for i in sub:
    temp = df_total[df_total.subject == i].reset_index(drop=True)
    temp.to_csv('/home/yuchen/shapes/notebooks/' + i + '.csv')
df_total.to_csv('/home/yuchen/shapes/notebooks/out.csv')

In [ ]:
copy = df_total.copy()
df_AllSub = df_total.copy()
df_s = pd.DataFrame({})
for subj in ['12-005','51-009','52-001','All']:
    if subj=='All':
        df_t = df_AllSub.copy()
    else:   
        df_t = copy[copy.subject == subj]
    df_t['amp1'] = (df_t['amp1'] - df_t['amp1'].mean()) / df_t['amp1'].std()
    df_t['freq'] = (df_t['freq'] - df_t['freq'].mean()) / df_t['freq'].std()
    df_t['distance_to_implant'] = (df_t['distance_to_implant'] - df_t['distance_to_implant'].mean()) / df_t['distance_to_implant'].std()
    df_t['distance_to_fovea'] = (df_t['distance_to_fovea'] - df_t['distance_to_fovea'].mean()) / df_t['distance_to_fovea'].std()
    if subj == 'All':  
        df_AllSub = df_t.copy()
    else:
        df_s = pd.concat([df_s, df_t])
        df_s = df_s.fillna(0)
        
df_total = df_s.copy().reset_index(drop=True)

In [ ]:
fig, axs= plt.subplots(ncols=4, figsize=(56, 14))
column_lst = ['amp1', 'freq','distance_to_fovea', 'distance_to_implant']
name_lst = ['Amplitude','Frequency','Foveal Eccentricity', 'Electrode-Implant Distance']
for dv in range(4):
    reg = LinearRegression().fit(df_total[column_lst[:dv] + column_lst[dv+1 :]], df_total[['area']])
    y_predicted = reg.predict(df_total[column_lst[:dv] + column_lst[dv+1 :]])
    y = df_total[['area']]-y_predicted

    reg = LinearRegression().fit(df_total[column_lst[:dv] + column_lst[dv+1 :]], df_total[column_lst[dv]])
    y_predicted = reg.predict(df_total[column_lst[:dv] + column_lst[dv+1 :]])
    x = df_total[column_lst[dv]]-y_predicted
    df_total['x'] = x
    df_total['y'] = y
    subject = ['12-005','51-009','52-001']
    marker_lst = ['o','s','^']
    color_lst = ['#1E88E5', '#FFC107','#004D40']
    for i in range(len(subject)):
        temp = df_total[df_total.subject == subject[i]]
        axs[dv].plot(temp['x'],temp['y'],marker_lst[i], color=color_lst[i],alpha=0.6,markersize=20)
    axs[dv].legend(subject, loc='upper right',prop={'size': 30})
    reg = LinearRegression().fit(np.array(x).reshape(-1,1),np.array(y).reshape(-1,1))
    y_pred = reg.predict(np.array(x).reshape(-1,1))
    axs[dv].plot(x, y_pred,'-', color="black",linewidth=2)
    # axs[dv].text(x=x.min(),y=y.max(), s='r = '+ str(np.corrcoef(df_total.x,df_total.y)[0,1].round(3)))
    for item in ([axs[dv].xaxis.label, axs[dv].yaxis.label]+axs[dv].get_xticklabels()+axs[dv].get_yticklabels()):
        item.set_fontsize(30)
    axs[dv].set(xlabel = name_lst[dv], ylabel = 'Number of Phopshene',ylim=(-0.7,1.2))
fig.savefig('/home/yuchen/paper/11f. Single-Electrode Number of Phosphenes.pdf', transparent=True)

In [ ]:
for subj in ['12-005','51-009','52-001','AllSubjects']:
    temp = df_total[df_total.subject == subj].reset_index(drop=True)
    if subj == '12-005':
        X = temp[['amp1','freq','distance_to_fovea','distance_to_implant']]
    elif subj == 'AllSubjects':
        temp = df_AllSub.copy()
        X = temp[['amp1','freq','distance_to_fovea','distance_to_implant']]
    else:
        X = temp[['amp1','freq','distance_to_fovea']]
    y = temp.area
    X2 = sm.add_constant(X)
    est = sm.OLS(y, X2)
    est2 = est.fit()
    print('subject: ' + subj)
    print(est2.summary())
    print('\ncoefficient with more digits: ')
    print(est2.params)
    print('\nvariance inflation factor: ')
    print([[X.columns[i],variance_inflation_factor(X.values, i)]
                          for i in range(len(X.columns))])
    print('\npartial correlation: ')
    X['area'] = temp['area']
    print(X.pcorr())
    print(' ')

## Correlation b/t distance, threshold, thickness, distance to fovea

In [ ]:
distance = [0,0,0,2,7,7,0,0,5,0,
                0,0,4,11,13,15,15,3,7,4,
                0,0,15,16,17,17,19,16,9,4,
                0,0,16,19,15,17,22,25,13,10,
                0,0,8,15,14,13,17,23,14,2,
                0,0,10,15,13,12,9,11,5,-1]

threshold = [20,24,52,81,145,129,153,97,81,65,
                 24,40,65,125,233,169,161,169,65,24,
                 36,0,145,218,177,145,117,97,0,36,
                 20,0,169,185,169,145,97,81,81,40,
                 28,36,93,169,169,169,129,93,36,44,
                 0,32,73,93,125,129,85,48,56,36]

thickness = [19,18,23,19,17,18,18,24,20,22,
             22,24,19,15,16,14,14,20,24,21,
             24,26,16,13,16,16,16,15,19,23,
             23,27,15,17,17,17,12,9,18,19,
             25,28,22,17,19,20,15,9,18,24,
             23,27,18,17,16,21,22,22,26,-1]


lst_d = []

s2 = shapes.subject_params['12-005']
implant,model = shapes.model_from_params(s2)
for i in string.ascii_uppercase[0:6]: 
    for j in range(1,11):
        electrode = i + str(j)
        lst_d.append(math.sqrt(implant[electrode].x**2 +implant[electrode].y**2 ))
            
df = pd.DataFrame({'distance':distance,
              'threshold':threshold,
             'thickness':thickness,
                  'distance_to_fovea':lst_d })

df = df[(df.distance!= -1) & (df.thickness != -1) & (df.threshold != 0)].reset_index(drop=True)



fig, (ax1, ax2) = plt.subplots(ncols=2, figsize=(15, 5))
p2p.viz.base.scatter_correlation(df.distance, df.threshold, ax = ax1)
p2p.viz.base.scatter_correlation(df.distance, df.thickness, ax=ax2)

In [ ]:
p2p.viz.base.scatter_correlation(df.threshold, df.thickness)

In [ ]:
df_temp = df[['distance','distance_to_fovea','threshold']]
df_temp = df_temp.drop_duplicates().reset_index(drop=True)
fig, (ax1, ax2) = plt.subplots(ncols=2, figsize=(15, 5))
p2p.viz.base.scatter_correlation(df_temp.distance_to_fovea, df_temp.threshold, ax = ax1)
p2p.viz.base.scatter_correlation(df_temp.distance_to_fovea, df_temp.distance, ax = ax2) 

In [ ]:
data = shapes.load_shapes("/home/yuchen/shapes/data/shapes.h5", subjects=['12-005','51-009','52-001'],stim_class=None)
data = data[data['electrode2'] == str()].reset_index(drop=True)

data.groupby(['subject']).count().reset_index()

fig, axes = plt.subplots(ncols=5, figsize=(15, 5))

df = data[(data.subject == '12-005') & (data.amp1 == 1.5) & (data.freq == 20)]

p2p.viz.base.scatter_correlation(df_temp.distance_to_fovea, df_temp.threshold, ax = axes[0])


In [ ]:
data = shapes.load_shapes("/home/yuchen/shapes/data/shapes.h5", subjects=['12-005','51-009','52-001'],stim_class=None)
data = data[data['electrode2'] == str()].reset_index(drop=True)

xrange = []
yrange = []
for i in data.groupby(['subject']).count().reset_index().amp1:
    print(i)
xrange.extend([(-30,30)]*1032) 
xrange.extend([(-32.5,32.5)]*819) 
xrange.extend([(-32,32)]*875) 
yrange.extend([(-22.5,22.5)] * 1032)
yrange.extend([(-24.4,24.4)] * 819)
yrange.extend([(-24,24)] * 875)

data['xrange'] = xrange
data['yrange'] = yrange
data = data.replace({'12-005':'S2', '51-009':'S3', '52-001':'S4'})
data = data.rename(columns={"electrode1": "electrode",'amp1':'amp'})
data = data[['subject','freq','electrode','amp','xrange','yrange','image']]

distance = [0,0,0,2,7,7,0,0,5,0,
                0,0,4,11,13,15,15,3,7,4,
                0,0,15,16,17,17,19,16,9,4,
                0,0,16,19,15,17,22,25,13,10,
                0,0,8,15,14,13,17,23,14,2,
                0,0,10,15,13,12,9,11,5,-1]

threshold = [20,24,52,81,145,129,153,97,81,65,
                 24,40,65,125,233,169,161,169,65,24,
                 36,0,145,218,177,145,117,97,0,36,
                 20,0,169,185,169,145,97,81,81,40,
                 28,36,93,169,169,169,129,93,36,44,
                 0,32,73,93,125,129,85,48,56,36]

thickness = [19,18,23,19,17,18,18,24,20,22,
             22,24,19,15,16,14,14,20,24,21,
             24,26,16,13,16,16,16,15,19,23,
             23,27,15,17,17,17,12,9,18,19,
             25,28,22,17,19,20,15,9,18,24,
             23,27,18,17,16,21,22,22,26,-1]


lst_d = []
lst_e = []
lst_s = []
subject_list = ['12-005']
data = data[data.subject == "S2"].reset_index(drop=True)

for subj in range(len(subject_list)):
    s2 = shapes.subject_params[subject_list[subj]]
    implant,model = shapes.model_from_params(s2)
    for i in string.ascii_uppercase[0:6]: 
        for j in range(1,11):
            electrode = i + str(j)
            lst_d.append(math.sqrt(implant[electrode].x**2 +implant[electrode].y**2 ))
            lst_e.append(electrode)
            lst_s.append('S' + str(subj+2))
            
df = pd.DataFrame({'electrode':lst_e,
                   'subject':lst_s,
                   'distance':distance,
                   'threshold':threshold,
                  'distance_to_fovea':lst_d })

df = df.merge(data, how = 'inner', on = ['electrode','subject'])
df

In [ ]:
argus = ArgusII(x=-1896, y =-542, rot=-44, eye='RE')
plot_argus_phosphenes(df, argus,scale=1)

In [ ]:
df.distance_to_fovea.max()

## Plot

In [ ]:
fig, axes = plt.subplots(ncols=3, nrows = 4, figsize=(20, 15))

argus = ArgusII(x=-1896, y =-542, rot=-44, eye='RE')

df_temp = df[(df.freq==20) & (df.amp==1.25) & (df.distance <= 8)]
lst_e = df_temp.electrode.tolist()
plot_argus_phosphenes(df_temp, argus,scale=1,ax = axes[0,0])
df_temp = df[(df.freq==20) & (df.amp==1.5) & (df.distance <= 8) & (df.electrode.isin(lst_e))]
plot_argus_phosphenes(df_temp, argus,scale=1,ax = axes[0,1])
df_temp = df[(df.freq==20) & (df.amp==2) & (df.distance <= 8) & (df.electrode.isin(lst_e))]
plot_argus_phosphenes(df_temp, argus,scale=1,ax = axes[0,2])

axes[0,0].set( title = 'Amplitude = 1.25xTh')
axes[0,1].set( title = 'Amplitude = 1.5xTh')
axes[0,2].set( title = 'Amplitude = 2xTh')

df_temp = df[(df.freq==6) & (df.amp==1.5)]
plot_argus_phosphenes(df_temp, argus,scale=1,ax = axes[1,0])
df_temp = df[(df.freq==40) & (df.amp==1.5)]
plot_argus_phosphenes(df_temp, argus,scale=1,ax = axes[1,1])
df_temp = df[(df.freq==120) & (df.amp==1.5)]
plot_argus_phosphenes(df_temp, argus,scale=1,ax = axes[1,2])

axes[1,0].set( title = 'Frequency = 6Hz')
axes[1,1].set( title = 'Frequency = 40Hz')
axes[1,2].set( title = 'Frequency = 120Hz')

df_temp = df[(df.freq==20) & (df.amp==1.5) & (df.distance_to_fovea <= 1500)]
plot_argus_phosphenes(df_temp, argus,scale=1,ax = axes[2,0])
df_temp = df[(df.freq==20) & (df.amp==1.5) & (df.distance_to_fovea > 1500) & (df.distance_to_fovea <= 3000)]
plot_argus_phosphenes(df_temp, argus,scale=1,ax = axes[2,1])
df_temp = df[(df.freq==20) & (df.amp==1.5) & (df.distance_to_fovea > 3000)]
plot_argus_phosphenes(df_temp, argus,scale=1,ax = axes[2,2])

axes[2,0].set( title = 'Electrode-Fovea Distance ≤ 1500 microns')
axes[2,1].set( title = '1500 microns < Electrode-Fovea Distance ≤ 3000 microns')
axes[2,2].set( title = '3000 microns < Electrode-Fovea Distance')

df_temp = df[(df.freq==20) & (df.amp==1.5) & (df.distance <= 8)]
plot_argus_phosphenes(df_temp, argus,scale=1,ax = axes[3,0])
df_temp = df[(df.freq==20) & (df.amp==1.5) & (df.distance > 8 ) & (df.distance <= 16)]
plot_argus_phosphenes(df_temp, argus,scale=1,ax = axes[3,1])
df_temp = df[(df.freq==20) & (df.amp==1.5) & (df.distance > 16)]
plot_argus_phosphenes(df_temp, argus,scale=1,ax = axes[3,2])

axes[3,0].set( title = 'Electrode-Retina Distance ≤ 8 pixels')
axes[3,1].set( title = '8 pixels < Electrode-Retina Distance ≤ 16 pixels')
axes[3,2].set( title = '16 pixels < Electrode-Retina Distance')

In [ ]:
fig, axes = plt.subplots(ncols=3, nrows = 1, figsize=(15, 5))

df_temp = df[(df.freq==20) & (df.amp==1.5) & (df.distance_to_fovea <= 1500)]
plot_argus_phosphenes(df_temp, argus,scale=1,ax = axes[0])
df_temp = df[(df.freq==20) & (df.amp==1.5) & (df.distance_to_fovea > 1500) & (df.distance_to_fovea <= 3000)]
plot_argus_phosphenes(df_temp, argus,scale=1,ax = axes[1])
df_temp = df[(df.freq==20) & (df.amp==1.5) & (df.distance_to_fovea > 3000)]
plot_argus_phosphenes(df_temp, argus,scale=1,ax = axes[2])

axes[0].set( title = 'distance <= 8pixels')
axes[1].set( title = '8pixels < distance <= 16pixels')
axes[2].set( title = '16pixels < distance')

In [ ]:
fig, axes = plt.subplots(ncols=3, nrows = 1, figsize=(15, 5))

df_temp = df[(df.freq==20) & (df.amp==2) & (df.distance == 0)]
plot_argus_phosphenes(df_temp, argus,scale=1,ax = axes[0])
df_temp = df[(df.freq==20) & (df.amp==2) & (df.distance > 0) & (df.distance <= 12)]
plot_argus_phosphenes(df_temp, argus,scale=1,ax = axes[1])
df_temp = df[(df.freq==20) & (df.amp==2) & (df.distance >= 12)]
plot_argus_phosphenes(df_temp, argus,scale=1,ax = axes[2])

axes[0].set( title = 'distance <= 8pixels')
axes[1].set( title = '8pixels < distance <= 16pixels')
axes[2].set( title = '16pixels < distance')

In [ ]:
data = shapes.load_shapes("/home/yuchen/shapes/data/shapes.h5", subjects=['12-005','51-009','52-001'],stim_class=None)
data = data[data['electrode2'] == str()].reset_index(drop=True)

df_total = pd.DataFrame()
sub = ['12-005','51-009','52-001']
sub = ['12-005']

for subj in sub:
    df = data[['subject','amp1','freq','electrode1']].drop_duplicates().reset_index(drop=True)
    
    df = df[df.subject == subj].reset_index(drop=True)
    lst = []
    lst_area = []
    for i in range(len(df)):
        df_temp = data[(data.amp1 == df.amp1[i]) & (data.freq == df.freq[i]) & (data.subject == df.subject[i]) & (data.electrode1 == df.electrode1[i])].reset_index(drop=True)
        num = len(df_temp[df_temp.num_regions > 1]) / len(df_temp)
        lst.append(num)
        if num == 0:
            lst_area.append(df_temp.orientation.mean())

        # if there are more than one phosphene in a drawing, pick the phosphene whose size, orientation, and eccentricity is closer to the average size, orientation, and eccentricity in that group
        elif num <= 0.5:
            avg_area = df_temp[df_temp.num_regions == 1][['area']].mean().tolist()[0]
            avg_orientation = df_temp[df_temp.num_regions == 1][['orientation']].mean().tolist()[0]
            avg_eccentricity = df_temp[df_temp.num_regions == 1][['eccentricity']].mean().tolist()[0]
            for item in range(len(df_temp)):
                if df_temp.num_regions[item] > 1:
                        label_img = skimage.measure.label(df_temp['image'][item]>0)
                        regions = regionprops(label_img)
                        props = regionprops_table(label_img, properties=('centroid',
                                                                         'orientation',
                                                                         'eccentricity',
                                                                         'major_axis_length',
                                                                         'minor_axis_length',
                                                                         'area'))
                        df_similar = pd.DataFrame(props).astype('object')
                        area_id = 0
                        area_compare = 9999999
                        for j in range(len(df_similar)):
                            result_temp = (df_similar.iloc[:, 6].tolist()[j] - avg_area)**2 + (df_similar.iloc[:, 2].tolist()[j] - avg_area)**2 + (df_similar.iloc[:, 3].tolist()[j] - avg_area)**2
                            if result_temp < area_compare:
                                area_id = j
                                area_compare = result_temp
                        df_temp.at[item,'orientation'] = df_similar.iloc[:, 3].tolist()[area_id]
            lst_area.append(df_temp.orientation.mean())
        else:
            lst_area.append(0)

    df['helper'] = lst
    df['area'] = lst_area
    df = df[df.helper <= 0.5].reset_index(drop = True)
    df = df.drop(columns = ['helper'])
    df = df.groupby(['subject','amp1','freq','electrode1']).mean().reset_index()

    reference_number = df[(df.subject == subj) & (df.amp1 == 1.5) & (df.freq == 20)].area.mean()
    for i in range(len(df)):
        df.at[i, 'area'] = df.at[i, 'area'] / reference_number


    # from https://docs.google.com/document/d/1piV1pldCefA13s4Qj3MbntjhTvRgotofoodQpyQn5iU/edit
    distance = [0,0,0,2,7,7,0,0,5,0,
                    0,0,4,11,13,15,15,3,7,4,
                    0,0,15,16,17,17,19,16,9,4,
                    0,0,16,19,15,17,22,25,13,10,
                    0,0,8,15,14,13,17,23,14,2,
                    0,0,10,15,13,12,9,11,5,-1]
    for i in range(len(distance)):
        distance[i]  = distance[i] * 22

    threshold = [20,24,52,81,145,129,153,97,81,65,
                     24,40,65,125,233,169,161,169,65,24,
                     36,0,145,218,177,145,117,97,0,36,
                     20,0,169,185,169,145,97,81,81,40,
                     28,36,93,169,169,169,129,93,36,44,
                     0,32,73,93,125,129,85,48,56,36]

    lst = []
    lst_d = []

    s2 = shapes.subject_params[subj]
    implant,model = shapes.model_from_params(s2)
    for i in string.ascii_uppercase[0:6]: 
        for j in range(1,11):
            electrode = i + str(j)
            lst.append(electrode)
            lst_d.append(math.sqrt(implant[electrode].x**2 +implant[electrode].y**2 ))

    df_o = pd.DataFrame(lst, columns=['electrode1'])
    df_o['distance'] = distance
    df_o['threshold'] = threshold
    df_o['distance_to_fovea'] = lst_d
    df_o = df_o[(df_o.distance > -1) & (df_o.threshold != 0)].reset_index(drop=True)

    df = df_o.merge(df, how = 'inner', on = 'electrode1')
    df = df.drop(columns = ['subject','electrode1'])
    df_total = pd.concat([df_total, df])
df_total.to_csv('/home/yuchen/shapes/notebooks/out.csv')